In [2]:
# 导入必要的库
import pandas as pd
import numpy as np

In [3]:
# 读取门店信息表数据
store_df = pd.read_excel('门店信息表.xlsx')

# 查看数据的基本信息
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 791 entries, 0 to 790
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   店铺代码    791 non-null    object 
 1   地区编码    791 non-null    object 
 2   渠道大类    791 non-null    object 
 3   商圈等级描述  764 non-null    object 
 4   商圈类别描述  749 non-null    object 
 5   店铺位置    748 non-null    object 
 6   店铺状态    791 non-null    object 
 7   仓储类别    791 non-null    object 
 8   营业员数    784 non-null    float64
 9   仓储面积    673 non-null    object 
 10  店铺等级    765 non-null    object 
 11  店铺时尚度   773 non-null    float64
 12  接受价格等级  757 non-null    object 
 13  店铺类型    768 non-null    float64
 14  装修代数    703 non-null    object 
dtypes: float64(3), object(12)
memory usage: 92.8+ KB


In [4]:
# 查看数据的前几行
store_df.head()

,店铺代码,地区编码,渠道大类,商圈等级描述,商圈类别描述,店铺位置,店铺状态,仓储类别,营业员数,仓储面积,店铺等级,店铺时尚度,接受价格等级,店铺类型,装修代数
0,DPX00X,GBL6020,DZ01,流行,中心,1F中岛,A,店外仓,2.0,15,A,2.0,中,1.0,8代
1,DPX002,GBL6020,DZ01,流行,中心,1F中岛,A,店外仓,2.0,17,A,1.0,中,1.0,7代
2,DPX003,GBL6020,DZ01,流行,中心,2F中岛,A,店外仓,2.0,10,B,2.0,中,1.0,7代
3,DPX004,GBL6020,DZ01,流行,新兴,1F边厅,A,店外仓,2.0,13,B,2.0,中,1.0,7代
4,DPX005,GBL6020,DZ01,流行,中心,1F边厅,A,店外仓,3.0,9,B,1.0,中,1.0,7代


In [5]:
# 创建一个计算缺失率的函数
def missing_rate(df):
    """计算每列的缺失率"""
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_stats = pd.DataFrame({'缺失值数量': missing, '缺失率(%)': missing_percent})
    return missing_stats.sort_values('缺失率(%)', ascending=False)

# 计算并显示各列缺失率
missing_rate(store_df)

,缺失值数量,缺失率(%)
仓储面积,118,14.917826
装修代数,88,11.125158
店铺位置,43,5.436157
商圈类别描述,42,5.309735
接受价格等级,34,4.298357
商圈等级描述,27,3.413401
店铺等级,26,3.286979
店铺类型,23,2.907712
店铺时尚度,18,2.275601
营业员数,7,0.884956


In [7]:
# 检查仓储面积列（可能应该是数字但当前是字符类型）
print("仓储面积的唯一值:")
store_df['仓储面积'].value_counts()

仓储面积的唯一值:


仓储面积
10       134
15        86
20        70
12        67
8         52
        ... 
8㎡         1
87         1
10平        1
12.2㎡      1
33.4       1
Name: count, Length: 98, dtype: int64

以看到仓储面积列存在格式不一致的问题，有的是纯数字，有的带有单位"㎡"或"平"

In [8]:
# 处理仓储面积列，提取数字部分
def extract_number(value):
    if pd.isna(value):
        return np.nan
    # 将值转为字符串
    value = str(value)
    # 使用正则表达式提取数字部分
    import re
    matches = re.findall(r'(\d+\.?\d*)', value)
    return float(matches[0]) if matches else np.nan

# 应用函数并替换原列
store_df['仓储面积_数值'] = store_df['仓储面积'].apply(extract_number)

# 检查转换结果的统计信息
print("转换后的仓储面积统计信息:")
store_df['仓储面积_数值'].describe()

转换后的仓储面积统计信息:


count    673.000000
mean      13.647103
std        6.844614
min        2.900000
25%       10.000000
50%       12.000000
75%       15.000000
max       87.000000
Name: 仓储面积_数值, dtype: float64

In [9]:
# 查看分类列的值分布
categorical_columns = ['店铺代码', '地区编码', '渠道大类', '商圈等级描述', '商圈类别描述', 
                      '店铺位置', '店铺状态', '仓储类别', '店铺等级', '接受价格等级', '装修代数']

for col in categorical_columns[:3]:  # 先看前三列
    print(f"\n{col}的值分布:")
    print(store_df[col].value_counts().head(10))  # 只显示前10个最常见的值


店铺代码的值分布:
店铺代码
DPJ04X    1
DPX00X    1
DPX002    1
DPX003    1
DPX004    1
DPX005    1
DPX006    1
DPJ0X4    1
DPJ0X3    1
DPJ0X2    1
Name: count, dtype: int64

地区编码的值分布:
地区编码
GBL6070    117
GBL6030     79
GBL6130     77
GBL6080     74
GBL6090     67
GBL6140     56
GBL6110     49
GBL6040     47
GBL6020     43
GBL6100     41
Name: count, dtype: int64

渠道大类的值分布:
渠道大类
DZ01    784
DZ02      6
DZ03      1
Name: count, dtype: int64


店铺代码是唯一的，每个值只出现一次，而地区编码和渠道大类有一定的分布。继续查看剩余的分类列：

In [10]:
# 继续查看其他分类列
for col in categorical_columns[3:7]:  # 查看下一组4列
    print(f"\n{col}的值分布:")
    print(store_df[col].value_counts().head(10))


商圈等级描述的值分布:
商圈等级描述
大众    417
流行    216
折扣     71
高端     50
无      10
Name: count, dtype: int64

商圈类别描述的值分布:
商圈类别描述
中心    382
社区    197
郊区    108
新兴     62
Name: count, dtype: int64

店铺位置的值分布:
店铺位置
1F边厅     428
2F边厅      89
1F中岛      84
3F边厅      24
1F次边厅     22
2F中岛      17
中岛        14
3F中岛      13
负1F边厅     13
边柜        10
Name: count, dtype: int64

店铺状态的值分布:
店铺状态
A     669
Z1     93
B      28
Z2      1
Name: count, dtype: int64


In [11]:
# 查看最后几个分类列
for col in categorical_columns[7:]:  # 查看最后几列
    print(f"\n{col}的值分布:")
    print(store_df[col].value_counts().head(10))


仓储类别的值分布:
仓储类别
店外仓    684
店内仓    105
后背仓      1
背库       1
Name: count, dtype: int64

店铺等级的值分布:
店铺等级
B    330
C    254
A    140
D     41
Name: count, dtype: int64

接受价格等级的值分布:
接受价格等级
中    534
低    140
高     83
Name: count, dtype: int64

装修代数的值分布:
装修代数
7代      442
8代      184
6代       38
5代       17
商场统装     13
9代        5
2代        3
1代        1
Name: count, dtype: int64


In [ ]:
# 删除临时列，保留我们需要的列
if '仓储面积_数值' in store_df.columns:
    # 用转换后的数值列替换原始列
    store_df['仓储面积'] = store_df['仓储面积_数值']
    # 删除临时列
    store_df = store_df.drop(columns=['仓储面积_数值'])

# 查看最终数据的结构
print("最终数据结构:")
store_df.info()

# 导出清洗后的数据
store_df.to_excel('门店信息表_清洗后.xlsx', index=False)

# 确认导出完成
print("已成功导出清洗后的门店信息表")